In [1]:
import numpy as np
import csv
import json
import os
from collections import defaultdict

In [2]:
with open('./data/annotation/raw/class-descriptions-boxable.csv', 'r') as fh:
    desc_data_boxable = [l.split(',') for l in fh.read().splitlines()]
desc_data_boxable = {l[0].strip(): l[1].strip() for l in desc_data_boxable}

In [3]:
with open('./data/annotation/raw/class-descriptions.csv', 'r') as fh:
    desc_data = [l.split(',') for l in fh.read().splitlines()]
desc_data = {l[0].strip(): l[1].strip() for l in desc_data}

In [4]:
with open('./data/annotation/raw/classes_trainable.txt', 'r') as fh:
    classes_trainable = [l.strip() for l in fh.read().splitlines()]

In [5]:
db_type = 'train'

In [6]:
# image ids with bounding boxes
image_ids = set([])
fn_label = './data/annotation/raw/{}-annotations-human-imagelabels-boxable.csv'.format(db_type)
with open(fn_label, 'r', newline='') as fh:
    reader = csv.reader(fh)
    for ii, row in enumerate(reader):
        if ii < 1:
            print(row)
        iid = row[0].strip()
        image_ids.add(iid)

['ImageID', 'Source', 'LabelName', 'Confidence']


In [7]:
label_counter = {k: 0 for k in classes_trainable}
fn_label = './data/annotation/raw/{}-annotations-human-imagelabels.csv'.format(db_type)
with open(fn_label, 'r', newline='') as fh:
    reader = csv.reader(fh)
    for ii, row in enumerate(reader):
        if ii < 1:
            print(row)
        if row[0].strip() not in image_ids:
            continue
        lid = row[2].strip()
        if lid in desc_data_boxable:
            continue
        try:
            label_counter[lid] += 1
        except:
            pass

['ImageID', 'Source', 'LabelName', 'Confidence']


In [8]:
counts = list(label_counter.values())
pivot_val = np.sort(counts)[-899]

In [9]:
desc_data_additional = {}
for k, v in label_counter.items():
    if v >= pivot_val:
        desc_data_additional[k] = desc_data[k]

In [10]:
desc_data_boxable.update(desc_data_additional)

In [11]:
# we will train on these classes
with open('./data/annotation/classes_all.txt', 'w') as fh:
    fh.write('\n'.join(desc_data_boxable.keys()))